In [0]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path =  "/content/drive/My Drive/online_shoppers_intention.csv"

In [0]:
data = pd.read_csv(path)
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [0]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin

In [0]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [0]:
def impute_cat_num(df):
    """
    simple imputer to fill in missing values
    args :
        df : dataframe
    return :
        dataframe where missing values are imputed
    """
    imputed_df = DataFrameImputer().fit_transform(df)
    return imputed_df

In [0]:
data = impute_cat_num(data)

In [0]:
from xgboost import XGBRegressor

def model(data):
    """
    to get the model for a given dataset
    args : 
        data : dataset_Id
    return :
        trained : model
    """


    df = impute_cat_num(data)

    df = label_encoder(df)

    y = df.Revenue
    X = df.drop(['Revenue'], axis=1)


    feature_names = [i for i in X.columns]

    X_train, X_valid, y_train, y_valid = train_test_split(X, y)

    my_model = XGBRegressor(n_estimators=1500)
    my_model.fit(X_train, y_train, 
                early_stopping_rounds= 300,
                eval_metric="logloss",
                eval_set=[(X_valid, y_valid)],
                verbose=True)

    return my_model

In [0]:
model = model(data)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[06:46:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-logloss:0.62725
Will train until validation_0-logloss hasn't improved in 300 rounds.
[1]	validation_0-logloss:0.57308
[2]	validation_0-logloss:0.52774
[3]	validation_0-logloss:0.489635
[4]	validation_0-logloss:0.457094
[5]	validation_0-logloss:0.429094
[6]	validation_0-logloss:0.404959
[7]	validation_0-logloss:0.384018
[8]	validation_0-logloss:0.366073
[9]	validation_0-logloss:0.350731
[10]	validation_0-logloss:0.337165
[11]	validation_0-logloss:0.32523
[12]	validation_0-logloss:0.315221
[13]	validation_0-logloss:0.306081
[14]	validation_0-logloss:0.298009
[15]	validation_0-logloss:0.290993
[16]	validation_0-logloss:0.284819
[17]	validation_0-logloss:0.27968
[18]	validation_0-logloss:0.275168
[19]	validation_0-logloss:0.270765
[20]	validation_0-logloss:0.26744
[21]	validation_0-logloss:0.264456
[22]	validation_0-logloss:0.261614
[23]	validation

In [0]:
!pip install pdpbox

In [0]:
from pdpbox import pdp, get_dataset, info_plots

df = label_encoder(data)
model_features = list(df)

y = df.Revenue
X = df.drop(['Revenue'], axis=1)


feature_names = [i for i in X.columns]

feature_names

['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'Month',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType',
 'VisitorType',
 'Weekend']

In [0]:
pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=feature_names, feature='Revenue')

# go to the github page of pdpbox and get the information

In [0]:
data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [0]:
def get_col_missing_values(df):
    """
    get number of columns with missing values and 
    how many missing values each column has
    args :
        df : dataframe
    return :
        column names with missing values with number of missing values
    """
    # Number of missing values in each column of dataframe
    missing_val_count_by_column = (df.isnull().sum())
    return missing_val_count_by_column[missing_val_count_by_column > 0]

In [0]:
print(get_col_missing_values(data))

Series([], dtype: int64)


In [0]:
def oh_encoder(df):
    """
    function to one hot encode the values to a way that final column name have a meaning
    args : 
        dataframe
    return :
        dataframe that one hot encoded
    """
    oh_df = df.copy()
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    for col in object_cols:
        val = oh_df[col].unique()
        for each in val :
            oh_df[f'{each}_{col}'] = oh_df[col].apply(lambda x : 1 if x == each else 0)
        oh_df = oh_df.drop([col], axis =1)
    return oh_df

In [0]:
def label_encoder(df):
    """
    function to label encode the values to a way that final column name have a meaning
    args : 
        dataframe
    return :
        dataframe that label encoded
    """
    
    label_df = df.copy()
    s = (df.dtypes == 'object')
    object_cols = list(s[s].index)
    labelEncoder = LabelEncoder()
    for col in object_cols:
        label_df[col] = labelEncoder.fit_transform(df[col])

    return label_df

In [0]:
oh_df = oh_encoder(data)

In [0]:
label_df = label_encoder(data)

In [0]:
oh_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType,Weekend,Revenue,Feb_Month,Mar_Month,May_Month,Oct_Month,June_Month,Jul_Month,Aug_Month,Nov_Month,Sep_Month,Dec_Month,Returning_Visitor_VisitorType,New_Visitor_VisitorType,Other_VisitorType
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,1,1,1,1,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,1,2,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,4,1,9,3,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,3,2,2,4,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,3,3,1,4,True,False,1,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
label_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,2,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,2,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,2,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,2,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,2,True,False


In [0]:
data.nunique()

Administrative               27
Administrative_Duration    3335
Informational                17
Informational_Duration     1258
ProductRelated              311
ProductRelated_Duration    9551
BounceRates                1872
ExitRates                  4777
PageValues                 2704
SpecialDay                    6
Month                        10
OperatingSystems              8
Browser                      13
Region                        9
TrafficType                  20
VisitorType                   3
Weekend                       2
Revenue                       2
dtype: int64

In [0]:
list(data)

['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'SpecialDay',
 'Month',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType',
 'VisitorType',
 'Weekend',
 'Revenue']

In [0]:
data['Month'].nunique()

10

In [0]:
def get_cat_features(df, support = 20):
  list_col = list(df)
  cat_col = []
  for i in list_col:
    if df[i].nunique() < support :
      cat_col.append(i)
  return cat_col
  

In [0]:
cat_col = get_cat_features(data)

In [0]:
cat_col

['Informational',
 'SpecialDay',
 'Month',
 'OperatingSystems',
 'Browser',
 'Region',
 'VisitorType',
 'Weekend',
 'Revenue']

In [0]:
def oh_encoder(df, cat_cols):
    """
    function to one hot encode the values to a way that final column name have a meaning
    args : 
        dataframe
    return :
        dataframe that one hot encoded
    """
    oh_df = df.copy()
    for col in cat_cols:
        val = oh_df[col].unique()
        for each in val :
            oh_df[f'{each}_{col}'] = oh_df[col].apply(lambda x : 1 if x == each else 0)
        oh_df = oh_df.drop([col], axis =1)
    return oh_df

In [0]:
cat_oh = oh_encoder(data, cat_col)

In [0]:
cat_oh.head()

,Administrative,Administrative_Duration,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,TrafficType,0_Informational,1_Informational,2_Informational,4_Informational,16_Informational,5_Informational,3_Informational,14_Informational,6_Informational,12_Informational,7_Informational,9_Informational,10_Informational,8_Informational,11_Informational,24_Informational,13_Informational,0.0_SpecialDay,0.4_SpecialDay,0.8_SpecialDay,1.0_SpecialDay,0.2_SpecialDay,0.6_SpecialDay,Feb_Month,Mar_Month,May_Month,Oct_Month,June_Month,Jul_Month,Aug_Month,Nov_Month,Sep_Month,Dec_Month,1_OperatingSystems,2_OperatingSystems,4_OperatingSystems,3_OperatingSystems,7_OperatingSystems,6_OperatingSystems,8_OperatingSystems,5_OperatingSystems,1_Browser,2_Browser,3_Browser,4_Browser,5_Browser,6_Browser,7_Browser,10_Browser,8_Browser,9_Browser,12_Browser,13_Browser,11_Browser,1_Region,9_Region,2_Region,3_Region,4_Region,5_Region,6_Region,7_Region,8_Region,Returning_Visitor_VisitorType,New_Visitor_VisitorType,Other_VisitorType,False_Weekend,True_Weekend,False_Revenue,True_Revenue
0,0,0.0,0.0,1,0.000000,0.20,0.20,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
1,0,0.0,0.0,2,64.000000,0.00,0.10,0.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
2,0,0.0,0.0,1,0.000000,0.20,0.20,0.0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0
3,0,0.0,0.0,2,2.666667,0.05,0.14,0.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0
4,0,0.0,0.0,10,627.500000,0.02,0.05,0.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0


In [0]:
cat_oh.shape

(12330, 79)

In [0]:
def oh_label_encoder(df, support = 10):
  encoded_df = label_encoder(df)
  cat_cols = get_cat_features(encoded_df, support)
  encoded_df = oh_encoder(encoded_df, cat_cols)
  return encoded_df

In [0]:
encoded_df = oh_label_encoder(data)
encoded_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Month,Browser,TrafficType,0.0_SpecialDay,0.4_SpecialDay,0.8_SpecialDay,1.0_SpecialDay,0.2_SpecialDay,0.6_SpecialDay,1_OperatingSystems,2_OperatingSystems,4_OperatingSystems,3_OperatingSystems,7_OperatingSystems,6_OperatingSystems,8_OperatingSystems,5_OperatingSystems,1_Region,9_Region,2_Region,3_Region,4_Region,5_Region,6_Region,7_Region,8_Region,2_VisitorType,0_VisitorType,1_VisitorType,False_Weekend,True_Weekend,False_Revenue,True_Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,2,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,2,2,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,2,1,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,2,2,4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,2,3,4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0


In [0]:
!pip install category_encoders

     |████████████████████████████████| 102kB 2.2MB/s 


In [0]:
import category_encoders as ce
from sklearn import metrics

In [0]:
def count_encoder(df, cat_cols):
  label_df = label_encoder(df)
  count_enc = ce.CountEncoder()
  count_encoded = count_enc.fit_transform(df[cat_cols])
  count_df = label_df.join(count_encoded.add_suffix("_count"))
  return count_df

In [0]:
def target_enc(df, cat_cols, label):
  target_df = df.copy()
  target_enc = ce.TargetEncoder(cols=cat_cols)
  target_enc.fit(df[cat_cols], df[label])
  target_df = target_df.join(target_enc.transform(df[cat_cols]).add_suffix('_target'))
  return target_df

In [0]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [0]:

def base_model(X_train, X_test, y_train, y_test):
  dtrain = lgb.Dataset(X_train, label=y_train)
  dvalid = lgb.Dataset(X_test, label=y_test)

  param = {'num_leaves': 64, 'objective': 'binary'}
  param['metric'] = 'auc'
  num_round = 2500
  bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=200, verbose_eval=False)
  valid_pred = bst.predict(X_test)
  valid_score = metrics.roc_auc_score(y_test, valid_pred)
  print(f"Validation AUC score: {valid_score:.4f}")
  return valid_score
  

In [0]:
def get_feature_cols(df, label):
  feature_cols = df.columns.drop(label)
  return feature_cols

In [0]:
def data_split(X, y):
  X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)
  return X_train, X_test, y_train, y_test 

In [0]:
def get_X_y(df, label):
  feature_cols = get_feature_cols(df, label)
  return df[feature_cols], df[label]

In [0]:
def evaluate_df(X, y):
  X_train, X_test, y_train, y_test = data_split(X, y)
  score = base_model(X_train, X_test, y_train, y_test)
  return score

In [0]:
label_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,2,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,2,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,2,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,2,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,2,True,False


In [0]:
label_X, y = get_X_y(label_df, 'Revenue')
score = evaluate_df(label_X, y)

Validation AUC score: 0.9307


In [0]:
oh_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType,Weekend,Revenue,Feb_Month,Mar_Month,May_Month,Oct_Month,June_Month,Jul_Month,Aug_Month,Nov_Month,Sep_Month,Dec_Month,Returning_Visitor_VisitorType,New_Visitor_VisitorType,Other_VisitorType
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,1,1,1,1,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,1,2,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,4,1,9,3,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,3,2,2,4,False,False,1,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,3,3,1,4,True,False,1,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
oh_X, y = get_X_y(oh_df, 'Revenue')
score = evaluate_df(oh_X,y)

Validation AUC score: 0.9305


In [0]:
X, y = get_X_y(data, 'Revenue')
cat_col = get_cat_features(X)
cat_oh = oh_encoder(X, cat_col)

In [0]:
cat_oh.head()

,Administrative,Administrative_Duration,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,TrafficType,0_Informational,1_Informational,2_Informational,4_Informational,16_Informational,5_Informational,3_Informational,14_Informational,6_Informational,12_Informational,7_Informational,9_Informational,10_Informational,8_Informational,11_Informational,24_Informational,13_Informational,0.0_SpecialDay,0.4_SpecialDay,0.8_SpecialDay,1.0_SpecialDay,0.2_SpecialDay,0.6_SpecialDay,Feb_Month,Mar_Month,May_Month,Oct_Month,June_Month,Jul_Month,Aug_Month,Nov_Month,Sep_Month,Dec_Month,1_OperatingSystems,2_OperatingSystems,4_OperatingSystems,3_OperatingSystems,7_OperatingSystems,6_OperatingSystems,8_OperatingSystems,5_OperatingSystems,1_Browser,2_Browser,3_Browser,4_Browser,5_Browser,6_Browser,7_Browser,10_Browser,8_Browser,9_Browser,12_Browser,13_Browser,11_Browser,1_Region,9_Region,2_Region,3_Region,4_Region,5_Region,6_Region,7_Region,8_Region,Returning_Visitor_VisitorType,New_Visitor_VisitorType,Other_VisitorType,False_Weekend,True_Weekend
0,0,0.0,0.0,1,0.000000,0.20,0.20,0.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
1,0,0.0,0.0,2,64.000000,0.00,0.10,0.0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
2,0,0.0,0.0,1,0.000000,0.20,0.20,0.0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0
3,0,0.0,0.0,2,2.666667,0.05,0.14,0.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0
4,0,0.0,0.0,10,627.500000,0.02,0.05,0.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1


In [0]:
evaluate_df(cat_oh, y)

Validation AUC score: 0.9294


In [0]:
encoded_df = oh_label_encoder(X)
encoded_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,Month,Browser,TrafficType,0.0_SpecialDay,0.4_SpecialDay,0.8_SpecialDay,1.0_SpecialDay,0.2_SpecialDay,0.6_SpecialDay,1_OperatingSystems,2_OperatingSystems,4_OperatingSystems,3_OperatingSystems,7_OperatingSystems,6_OperatingSystems,8_OperatingSystems,5_OperatingSystems,1_Region,9_Region,2_Region,3_Region,4_Region,5_Region,6_Region,7_Region,8_Region,2_VisitorType,0_VisitorType,1_VisitorType,False_Weekend,True_Weekend
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,2,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,2,2,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,2,1,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,2,2,4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,2,3,4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1


In [0]:
evaluate_df(encoded_df, y)

Validation AUC score: 0.9307


In [0]:
def evaluate_pipeline_cat(data, label):
  label_df = label_encoder(data)
  X, y = get_X_y(label_df, label)
  score_opt = evaluate_df(X, y)
  print(score_opt)
  optimal_support = 0
  support = 10
  for i in range(10):
    cat_col = get_cat_features(X,support = support)
    cat_oh = oh_encoder(X, cat_col)
    score = evaluate_df(cat_oh, y)
    if score > score_opt:
      score_opt = score
      optimal_support = support
    print(score)
    support += 5
  return optimal_support

In [0]:
support = evaluate_pipeline_cat(data, 'Revenue')
print(support)

Validation AUC score: 0.9307
0.9306904222000305
Validation AUC score: 0.9307
0.930705364811208
Validation AUC score: 0.9304
0.9304329495151238
Validation AUC score: 0.9294
0.9293823690061739
Validation AUC score: 0.9287
0.9287363884306489
Validation AUC score: 0.9302
0.9302161667098852
Validation AUC score: 0.9302
0.9302161667098852
Validation AUC score: 0.9302
0.9302161667098852
Validation AUC score: 0.9302
0.9302161667098852
Validation AUC score: 0.9302
0.9302161667098852
Validation AUC score: 0.9302
0.9302161667098852
60


In [0]:
def count_encoder(df, cat_cols):
  label_df = label_encoder(df)
  count_enc = ce.CountEncoder()
  count_encoded = count_enc.fit_transform(df[cat_cols])
  count_df = label_df.join(count_encoded.add_suffix("_count"))
  return count_df

In [0]:
count_df = count_encoder(data, cat_col)

In [0]:
count_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue,Informational_count,SpecialDay_count,Month_count,OperatingSystems_count,Browser_count,Region_count,VisitorType_count,Weekend_count
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,2,False,False,0,0.0,184,1,1,1,10551,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,2,False,False,0,0.0,184,2,2,1,10551,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,2,False,False,0,0.0,184,4,1,9,10551,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,2,False,False,0,0.0,184,3,2,2,10551,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,2,True,False,0,0.0,184,3,3,1,10551,True


In [0]:
X, y = get_X_y(count_df, 'Revenue')

In [0]:
evaluate_df(X, y)

Validation AUC score: 0.9316


In [0]:
data.nunique()

Administrative               27
Administrative_Duration    3335
Informational                17
Informational_Duration     1258
ProductRelated              311
ProductRelated_Duration    9551
BounceRates                1872
ExitRates                  4777
PageValues                 2704
SpecialDay                    6
Month                        10
OperatingSystems              8
Browser                      13
Region                        9
TrafficType                  20
VisitorType                   3
Weekend                       2
Revenue                       2
dtype: int64

In [0]:
cat_feature = 

In [0]:
def get_cat_features_bounded(df, lower = 15, fraction = 3):
  lenght = len(df.index)
  upper = lenght / fraction
  list_col = list(df)
  cat_col = []
  for i in list_col:
    if df[i].nunique() > lower :
      if df[i].nunique() < upper :
        cat_col.append(i)
  return cat_col

In [0]:
X, y = get_X_y(data, 'Revenue')

In [0]:
cat_feature = get_cat_features_bounded(X)

In [0]:
cat_feature

['Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'BounceRates',
 'PageValues',
 'TrafficType']

In [0]:
count_df = count_encoder(data, cat_feature)

In [0]:
X_count, y = get_X_y(count_df, 'Revenue')

In [0]:
evaluate_df(X_count, y)

Validation AUC score: 0.9307


In [0]:
cat_feature = get_cat_features_bounded(X, lower = 5, fraction = 5)

In [0]:
count_df = count_encoder(data, cat_feature)

In [0]:
X_count, y = get_X_y(count_df, 'Revenue')

In [0]:
evaluate_df(X_count, y)

Validation AUC score: 0.9316


In [0]:
cat_feature = get_cat_features_bounded(X, lower = 5, fraction = 5)
count_df = count_encoder(data, cat_feature)
X_count, y = get_X_y(count_df, 'Revenue')

In [0]:
target_enc = ce.TargetEncoder(cols=cat_features)

train, valid, _ = get_data_splits(data)

# Fit the encoder using the categorical features and target
target_enc.fit(train[cat_features], train['outcome'])

# Transform the features, rename the columns with _target suffix, and join to dataframe
train = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
valid = valid.join(target_enc.transform(valid[cat_features]).add_suffix('_target')

In [0]:
def target_enc(X_train, df, cat_cols, y_train):
  target_df_tr = X_train.copy()
  target_df = df.copy()
  target_enc = ce.TargetEncoder(cols=cat_cols)
  target_enc.fit(target_df_tr[cat_cols], y_train)
  target_df = target_df.join(target_enc.transform(target_df[cat_cols]).add_suffix('_target'))
  return target_df

In [0]:
X, y = get_X_y(data, 'Revenue')

In [0]:
X_train, X_test, y_train, y_test = data_split(X, y)

target_df = target_enc(X_train, label_df, cat_col, y_train)

In [0]:
target_df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue,Informational_target,SpecialDay_target,Month_target,OperatingSystems_target,Browser_target,Region_target,VisitorType_target,Weekend_target
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,2,False,False,0.132482,0.1653,0.15434,0.154509,0.149910,0.159774,0.15434,0.149890
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,2,False,False,0.132482,0.1653,0.15434,0.172096,0.153962,0.159774,0.15434,0.149890
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,2,False,False,0.132482,0.1653,0.15434,0.176101,0.149910,0.163205,0.15434,0.149890
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,2,False,False,0.132482,0.1653,0.15434,0.102793,0.153962,0.166879,0.15434,0.149890
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,2,True,False,0.132482,0.1653,0.15434,0.102793,0.058824,0.159774,0.15434,0.169411


In [0]:
X_target, y =  get_X_y(target_df, 'Revenue')


In [0]:
evaluate_df(X_target, y)

Validation AUC score: 0.9299


In [0]:
def catboost_enc(X_train, df, cat_cols, y_train):
  target_df_tr = X_train.copy()
  target_df = df.copy()
  target_enc = ce.CatBoostEncoder(cols=cat_cols)
  target_enc.fit(target_df_tr[cat_cols], y_train)
  target_df = target_df.join(target_enc.transform(target_df[cat_cols]).add_suffix('_cb'))
  return target_df

In [0]:
X, y = get_X_y(data, 'Revenue')

X_train, X_test, y_train, y_test = data_split(X, y)

catboost_df = catboost_enc(X_train, label_df, cat_feature, y_train)

X_cat_boost, y =  get_X_y(catboost_df, 'Revenue')

In [0]:
X_cat_boost.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Administrative_cb,Informational_cb,Informational_Duration_cb,ProductRelated_cb,BounceRates_cb,SpecialDay_cb,Month_cb,OperatingSystems_cb,Browser_cb,Region_cb,TrafficType_cb
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,1,1,1,1,2,False,0.088605,0.132485,0.134309,0.025838,0.004841,0.165299,0.15434,0.154509,0.149912,0.159773,0.111205
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2,2,2,1,2,2,False,0.088605,0.132485,0.134309,0.048416,0.191285,0.165299,0.15434,0.172092,0.153962,0.159773,0.205616
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2,4,1,9,3,2,False,0.088605,0.132485,0.134309,0.025838,0.004841,0.165299,0.15434,0.176032,0.149912,0.163179,0.093594
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2,3,2,2,4,2,False,0.088605,0.132485,0.134309,0.048416,0.040440,0.165299,0.15434,0.102823,0.153962,0.166863,0.162432
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2,3,3,1,4,2,True,0.088605,0.132485,0.134309,0.138323,0.093248,0.165299,0.15434,0.102823,0.060208,0.159773,0.162432


In [0]:
evaluate_df(X_cat_boost, y)

Validation AUC score: 0.9074
